<a href="https://colab.research.google.com/github/TheSanjBot/underwater-image-enhancement/blob/main/uwenhancementv3nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

sanjayseven_uwenhancementv3_path = kagglehub.dataset_download('sanjayseven/uwenhancementv3')

print('Data source import complete.')


# CS517 Project - UnderWater Image Enhancement
## Pranjali Bajpai - 2018EEB1243
## Yogesh Vaidhya - 2018EEB1277


In [ ]:
# Import libraries
from PIL import Image

# Load image
image1 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img1.png')
image2 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img2.png')
image3 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img3.png')
image4 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img4.png')
image5 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img5.png')
image6 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img6.png')
image7 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img7.png')
image8 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img8.jpg')
image9 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img9.jpg')
image10 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/raw/Img10.jpg')
# Load reference images
rimage1 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/RImg1.png')
rimage2 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/RImg2.png')
rimage3 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/RImg3.png')
rimage4 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/RImg4.png')
rimage5 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/RImg5.png')
rimage6 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/RImg6.png')
rimage7 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/RImg7.png')
rimage8 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/Img8.jpg')
rimage9 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/Img9.jpg')
rimage10 = Image.open('/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/dataset/reference/Img10.jpg')

In [ ]:
!pip install cupy-cuda11x  # Install CuPy for Kaggle's CUDA 11.x

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import cv2
import torch
import numpy as np

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def rgb_to_hsv(image_tensor):
    """Convert RGB tensor to HSV space (PyTorch implementation)"""
    img = image_tensor / 255.0
    r, g, b = img[0], img[1], img[2]

    maxc, _ = torch.max(img, dim=0)
    minc, _ = torch.min(img, dim=0)
    delta = maxc - minc

    h = torch.zeros_like(maxc)
    s = torch.zeros_like(maxc)
    v = maxc

    mask = delta != 0
    rc = (maxc - r) / (delta + 1e-6)
    gc = (maxc - g) / (delta + 1e-6)
    bc = (maxc - b) / (delta + 1e-6)

    h = torch.where((maxc == r) & mask, bc - gc, h)
    h = torch.where((maxc == g) & mask, 2.0 + rc - bc, h)
    h = torch.where((maxc == b) & mask, 4.0 + gc - rc, h)
    h = (h / 6.0) % 1.0

    s = torch.where(maxc > 0, delta / maxc, 0.0)

    h = h * 179.0
    s = s * 255.0
    v = v * 255.0

    return torch.stack([h, s, v], dim=0)

def hsv_to_rgb(hsv_tensor):
    """Convert HSV tensor to RGB space (PyTorch implementation)"""
    h, s, v = hsv_tensor[0]/179.0, hsv_tensor[1]/255.0, hsv_tensor[2]/255.0

    c = v * s
    x = c * (1 - torch.abs((h * 6) % 2 - 1))
    m = v - c

    h_prime = (h * 6) % 6
    rgb = torch.zeros_like(hsv_tensor)

    cond = (h_prime >= 0) & (h_prime < 1)
    rgb[0] = torch.where(cond, c, rgb[0])
    rgb[1] = torch.where(cond, x, rgb[1])

    cond = (h_prime >= 1) & (h_prime < 2)
    rgb[0] = torch.where(cond, x, rgb[0])
    rgb[1] = torch.where(cond, c, rgb[1])

    cond = (h_prime >= 2) & (h_prime < 3)
    rgb[1] = torch.where(cond, c, rgb[1])
    rgb[2] = torch.where(cond, x, rgb[2])

    cond = (h_prime >= 3) & (h_prime < 4)
    rgb[1] = torch.where(cond, x, rgb[1])
    rgb[2] = torch.where(cond, c, rgb[2])

    cond = (h_prime >= 4) & (h_prime < 5)
    rgb[0] = torch.where(cond, x, rgb[0])
    rgb[2] = torch.where(cond, c, rgb[2])

    cond = (h_prime >= 5) & (h_prime < 6)
    rgb[0] = torch.where(cond, c, rgb[0])
    rgb[2] = torch.where(cond, x, rgb[2])

    rgb = (rgb + m) * 255.0
    return torch.clamp(rgb, 0.0, 255.0)

def compensate_RB_gpu(image_tensor, flag):
    """GPU-accelerated channel compensation"""
    R, G, B = image_tensor[0], image_tensor[1], image_tensor[2]

    minR, maxR = torch.min(R), torch.max(R)
    minG, maxG = torch.min(G), torch.max(G)
    minB, maxB = torch.min(B), torch.max(B)

    R_norm = (R - minR) / (maxR - minR + 1e-6)
    G_norm = (G - minG) / (maxG - minG + 1e-6)
    B_norm = (B - minB) / (maxB - minB + 1e-6)

    meanR, meanG, meanB = torch.mean(R_norm), torch.mean(G_norm), torch.mean(B_norm)

    if flag == 0:
        R_comp = (R_norm + (meanG - meanR) * (1 - R_norm) * G_norm) * maxR
        B_comp = (B_norm + (meanG - meanB) * (1 - B_norm) * G_norm) * maxB
        G_comp = G_norm * maxG
    else:
        R_comp = (R_norm + (meanG - meanR) * (1 - R_norm) * G_norm) * maxR
        B_comp = B_norm * maxB
        G_comp = G_norm * maxG

    return torch.stack([
        torch.clamp(R_comp, 0, 255),
        torch.clamp(G_comp, 0, 255),
        torch.clamp(B_comp, 0, 255)
    ], dim=0)

def gray_world_gpu(image_tensor):
    """GPU-accelerated gray world white balancing"""
    R, G, B = image_tensor[0], image_tensor[1], image_tensor[2]
    gray = 0.299 * R + 0.587 * G + 0.114 * B
    mean_gray = torch.mean(gray)

    R_new = R * (mean_gray / torch.mean(R))
    G_new = G * (mean_gray / torch.mean(G))
    B_new = B * (mean_gray / torch.mean(B))

    return torch.stack([
        torch.clamp(R_new, 0, 255),
        torch.clamp(G_new, 0, 255),
        torch.clamp(B_new, 0, 255)
    ], dim=0)

def equalize_gpu(v_channel):
    """GPU histogram equalization for V channel"""
    hist = torch.histc(v_channel, bins=256, min=0, max=255)
    cdf = torch.cumsum(hist, 0)
    cdf_min = cdf[cdf > 0].min()
    cdf = (cdf - cdf_min) / (v_channel.numel() - cdf_min + 1e-6)
    lut = (cdf * 255).clamp(0, 255)
    return lut[v_channel.to(torch.long).clamp(0, 255)]

def hsv_global_equalization_gpu(image_tensor):
    """GPU-accelerated HSV equalization"""
    hsv = rgb_to_hsv(image_tensor)
    v_eq = equalize_gpu(hsv[2].to(torch.float32))
    hsv_eq = torch.stack([hsv[0], hsv[1], v_eq], dim=0)
    return hsv_to_rgb(hsv_eq)

def sharpen_gpu(image_tensor):
    """GPU-accelerated sharpening"""
    kernel = torch.tensor([[1, 4, 6, 4, 1],
                          [4, 16, 24, 16, 4],
                          [6, 24, 36, 24, 6],
                          [4, 16, 24, 16, 4],
                          [1, 4, 6, 4, 1]], dtype=torch.float32, device=device) / 256.0
    kernel = kernel.repeat(3, 1, 1, 1)
    blurred = torch.nn.functional.conv2d(image_tensor.unsqueeze(0), kernel, padding=2, groups=3)
    return torch.clamp(1.5 * image_tensor - 0.5 * blurred.squeeze(0), 0, 255)


In [ ]:
def underwater_image_enhancement(image, flag=1, scale_factor=1.0):
    """
    Enhanced GPU-accelerated version of the original function
    Supports both PIL Images and numpy arrays
    """
    # Convert input to tensor
    if isinstance(image, Image.Image):
        pil_input = True
        image_np = np.array(image)
        if scale_factor != 1.0:
            new_size = (int(image.size[0]*scale_factor)), (int(image.size[1]*scale_factor))
            image_np = np.array(image.resize(new_size, Image.Resampling.LANCZOS))
    else:
        pil_input = False
        image_np = image.copy()
        if scale_factor != 1.0:
            h, w = image_np.shape[:2]
            new_size = (int(w*scale_factor), int(h*scale_factor))
            image_np = cv2.resize(image_np, new_size, interpolation=cv2.INTER_AREA)

    # Convert to tensor and process
    tensor = torch.from_numpy(image_np).permute(2, 0, 1).float().to(device)

    # Processing pipeline
    compensated = compensate_RB_gpu(tensor, flag)
    whitebalanced = gray_world_gpu(compensated)
    contrastenhanced = hsv_global_equalization_gpu(whitebalanced)
    sharpened = sharpen_gpu(whitebalanced)

    # Fusion
    fused = torch.clamp(0.5*contrastenhanced + 0.5*sharpened, 0, 255)

    # Convert back to original format
    output_np = fused.permute(1, 2, 0).byte().cpu().numpy()

    if pil_input:
        return Image.fromarray(output_np)
    return output_np

In [ ]:
averagefused1 = underwater_image_enhancement(image1, scale_factor=0.75)

averagefused1.save('/kaggle/working/averagefused1.png')

# For maximum speed (lowest quality)
enhance_video(input_path, output_path, scale_factor=0.4)

# For balanced quality/speed
enhance_video(input_path, output_path, scale_factor=0.67)

# For near-original quality
enhance_video(input_path, output_path, scale_factor=0.85)

In [ ]:
def process_frame_gpu(frame, flag=1):
    """GPU-accelerated frame processing pipeline"""
    tensor = torch.from_numpy(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) \
              .permute(2, 0, 1).float().to(device)

    compensated = compensate_RB_gpu(tensor, flag)
    whitebalanced = gray_world_gpu(compensated)
    contrastenhanced = hsv_global_equalization_gpu(whitebalanced)
    sharpened = sharpen_gpu(whitebalanced)

    fused = torch.clamp(0.5 * contrastenhanced + 0.5 * sharpened, 0, 255)
    return cv2.cvtColor(fused.permute(1, 2, 0).byte().cpu().numpy(), cv2.COLOR_RGB2BGR)

def enhance_video(input_path, output_path, flag=1, scale_factor=1.0):
    """Main video enhancement function with progress tracking"""
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    # Get video properties
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate scaled dimensions
    new_width = int(original_width * scale_factor)
    new_height = int(original_height * scale_factor)
    new_width -= new_width % 2
    new_height -= new_height % 2

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))

    current_frame = 0
    print(f"Total frames to process: {total_frames}")
    print("Progress: [", end='', flush=True)

    while True:
        ret, frame = cap.read()
        if not ret: break

        # Process frame
        frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_AREA)
        processed_frame = process_frame_gpu(frame, flag)
        out.write(processed_frame)

        # Update progress
        current_frame += 1
        progress = current_frame / total_frames
        if current_frame % max(1, total_frames//100) == 0:  # Update every 1% or 1 frame
            print("#", end='', flush=True)

    # Cleanup
    cap.release()
    out.release()
    print("]\nProcessing completed!")
    print(f"Enhanced video saved to: {output_path}")

# Example usage with progress tracking
enhance_video(
    "/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/GX016431.MP4",
    "/kaggle/working/output.mp4",
    flag=1,
    scale_factor=0.5
)

**code v2**

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageOps

def compensate_RB(image_np, flag):
    """Compensates Red and Blue channels in an image to improve underwater visibility."""
    image = Image.fromarray(image_np)  # Convert to PIL

    imager, imageg, imageb = image.split()
    minR, maxR = imager.getextrema()
    minG, maxG = imageg.getextrema()
    minB, maxB = imageb.getextrema()

    # Convert to NumPy arrays
    imageR = np.array(imager, np.float64)
    imageG = np.array(imageg, np.float64)
    imageB = np.array(imageb, np.float64)

    # Normalize (Avoid divide by zero)
    imageR = (imageR - minR) / (maxR - minR + 1e-6)
    imageG = (imageG - minG) / (maxG - minG + 1e-6)
    imageB = (imageB - minB) / (maxB - minB + 1e-6)

    # Get mean values
    meanR, meanG, meanB = np.mean(imageR), np.mean(imageG), np.mean(imageB)

    # Apply Compensation
    if flag == 0:
        imageR = (imageR + (meanG - meanR) * (1 - imageR) * imageG) * maxR
        imageB = (imageB + (meanG - meanB) * (1 - imageB) * imageG) * maxB
        imageG *= maxG

    elif flag == 1:
        imageR = (imageR + (meanG - meanR) * (1 - imageR) * imageG) * maxR
        imageB *= maxB
        imageG *= maxG

    # Convert back to uint8
    return np.stack([
        np.clip(imageR, 0, 255).astype(np.uint8),
        np.clip(imageG, 0, 255).astype(np.uint8),
        np.clip(imageB, 0, 255).astype(np.uint8)
    ], axis=-1)

def gray_world(image_np):
    """Applies the Gray World Algorithm for white balance."""
    image = Image.fromarray(image_np)

    imager, imageg, imageb = image.split()
    imageGray = image.convert('L')

    # Convert to NumPy
    imageR = np.array(imager, np.float64)
    imageG = np.array(imageg, np.float64)
    imageB = np.array(imageb, np.float64)
    meanGray = np.mean(np.array(imageGray, np.float64))

    # Mean Values
    meanR, meanG, meanB = np.mean(imageR), np.mean(imageG), np.mean(imageB)

    # Apply Gray World
    return np.stack([
        np.clip(imageR * meanGray / meanR, 0, 255).astype(np.uint8),
        np.clip(imageG * meanGray / meanG, 0, 255).astype(np.uint8),
        np.clip(imageB * meanGray / meanB, 0, 255).astype(np.uint8)
    ], axis=-1)

def hsv_global_equalization(image_np):
    """Applies Histogram Equalization in HSV space to enhance contrast."""
    hsvimage = cv2.cvtColor(image_np, cv2.COLOR_RGB2HSV)
    hsvimage[:, :, 2] = cv2.equalizeHist(hsvimage[:, :, 2])
    return cv2.cvtColor(hsvimage, cv2.COLOR_HSV2RGB)

def sharpen(image_np):
    """Sharpens the image using Gaussian Blur."""
    blurred = cv2.GaussianBlur(image_np, (5, 5), 1.0)
    return cv2.addWeighted(image_np, 1.5, blurred, -0.5, 0)




def underwater_image_enhancement(image, refimage, flag):
    # Convert PIL Image to numpy array
    if isinstance(image, Image.Image):
        pil_input = True
        image_np = np.array(image)
    else:
        pil_input = False
        image_np = image

    # Processing pipeline
    compensated = compensate_RB(image_np, flag)
    whitebalanced = gray_world(compensated)
    contrastenhanced = hsv_global_equalization(whitebalanced)
    sharpened = sharpen(whitebalanced)

    # Fusion and output conversion
    averagefused = cv2.addWeighted(contrastenhanced, 0.5, sharpened, 0.5, 0)

    # Convert back to PIL if input was PIL
    return Image.fromarray(averagefused) if pil_input else averagefused

In [ ]:
averagefused1 = underwater_image_enhancement(image1, None, 1)

averagefused1.save('/kaggle/working/averagefused1.png')

In [ ]:
def process_frame(frame, flag=1):
    """Processes a single frame through the underwater enhancement pipeline."""
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
    compensated = compensate_RB(frame, flag)
    whitebalanced = gray_world(compensated)
    contrastenhanced = hsv_global_equalization(whitebalanced)
    sharpened = sharpen(whitebalanced)

    # Fusion of contrast-enhanced and sharpened frames
    output_frame = cv2.addWeighted(contrastenhanced, 0.5, sharpened, 0.5, 0)

    return cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)  # Convert back to BGR

def enhance_video(input_path, output_path, flag=1):
    """Enhances an underwater video with progress tracking"""
    cap = cv2.VideoCapture(input_path)

    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    current_frame = 0
    print(f"Total frames to process: {total_frames}")
    print("Progress: [", end='', flush=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Stop if no more frames

        enhanced_frame = process_frame(frame, flag)
        out.write(enhanced_frame)

        # Update progress every 1% of frames
        current_frame += 1
        progress = current_frame / total_frames
        if current_frame % max(1, total_frames//100) == 0:
            print("#", end='', flush=True)

    cap.release()
    out.release()
    print("]\nProcessing completed!")
    print(f"Enhanced video saved to: {output_path}")

# Example usage
input_video = "/kaggle/input/uwenhancementv3/underwater-image-enhancement-main/GX016431.MP4"
output_video = "/kaggle/working/output1.mp4"
enhance_video(input_video, output_video, flag=1)